In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point

In [2]:
dfCrime = pd.read_excel('/Users/austincoffelt/Downloads/Rent-Walk-Local/NIBRSPublicView2019.xlsx')
print(dfCrime.shape)
dfCrime.head()

(260529, 16)


,Incident,RMSOccurrenceDate,RMSOccurrenceHour,NIBRSClass,NIBRSDescription,OffenseCount,Beat,Premise,StreetNo,StreetName,StreetType,Suffix,City,ZIPCode,MapLongitude,MapLatitude
0,5619,2019-01-01,0,290,"Destruction, damage, vandalism",1,9C30,"Residence, Home (Includes Apartment)",9622,SAN CARLOS,NaN,NaN,HOUSTON,77013,-95.258222,29.796918
1,17319,2019-01-01,0,35A,"Drug, narcotic violations",1,7C10,"Highway, Road, Street, Alley",NaN,EAST,FWY,NaN,HOUSTON,77020,-95.315170,29.774653
2,17319,2019-01-01,0,90D,Driving under the influence,1,7C10,"Highway, Road, Street, Alley",NaN,EAST,FWY,NaN,HOUSTON,77020,-95.315170,29.774653
3,18119,2019-01-01,0,290,"Destruction, damage, vandalism",1,16E40,"Residence, Home (Includes Apartment)",16718,LONE QUAIL,CT,NaN,HOUSTON,77489,-95.512372,29.583499
4,19019,2019-01-01,0,520,Weapon law violations,1,NaN,"Residence, Home (Includes Apartment)",1909,MELBOURNE,NaN,NaN,HOUSTON,77026-0000,-95.349516,29.816864


In [5]:
violentCrimes = ['Aggravated Assault', 'Forcible fondling', 'Forcible rape', 'Forcible sodomy', 'Murder, non-negligent',
                'Robbery']
dfCrime = dfCrime[dfCrime['NIBRSDescription'].isin(violentCrimes)]
print(dfCrime.shape)
dfCrime.head()

(24318, 16)


,Incident,RMSOccurrenceDate,RMSOccurrenceHour,NIBRSClass,NIBRSDescription,OffenseCount,Beat,Premise,StreetNo,StreetName,StreetType,Suffix,City,ZIPCode,MapLongitude,MapLatitude
6,20519,2019-01-01,0,13A,Aggravated Assault,1,15E30,"Residence, Home (Includes Apartment)",4034,OSBY,DR,NaN,HOUSTON,77025,-95.441322,29.676066
16,31119,2019-01-01,0,13A,Aggravated Assault,2,19G20,"Residence, Home (Includes Apartment)",4700,KIRKWOOD,RD,S,HOUSTON,77072,-95.587899,29.704853
23,40219,2019-01-01,0,13A,Aggravated Assault,1,13D20,"Residence, Home (Includes Apartment)",6502,ROXBURY,RD,NaN,HOUSTON,77087,-95.314375,29.671017
32,45919,2019-01-01,0,120,Robbery,1,1A30,"Highway, Road, Street, Alley",2200,SOUTHWEST,FWY,NaN,HOUSTON,77098,-95.412972,29.731193
33,47919,2019-01-01,0,120,Robbery,1,5F20,"Highway, Road, Street, Alley",8680,EMNORA,LN,NaN,HOUSTON,77080,-95.505826,29.815337


In [7]:
dfCrime = dfCrime[['NIBRSDescription', 'MapLatitude', 'MapLongitude']]

# make our lon and lat into a point type for spacial join
dfCrime['geometry'] = dfCrime.apply(lambda row: Point(row['MapLongitude'], row['MapLatitude']), axis=1)
dfCrime = dfCrime[['NIBRSDescription', 'geometry']]
dfCrime.head()

,NIBRSDescription,geometry
6,Aggravated Assault,POINT (-95.441322 29.676066)
16,Aggravated Assault,POINT (-95.58789899999999 29.704853)
23,Aggravated Assault,POINT (-95.314375 29.671017)
32,Robbery,POINT (-95.412972 29.731193)
33,Robbery,POINT (-95.505826 29.815337)


In [9]:
# Read shapefile (automatically loads .shp, .shx, .dbf)
gdf = gpd.read_file("/Users/austincoffelt/Downloads/Rent-Walk-Local/cb_2018_48_bg_500k/cb_2018_48_bg_500k.shp")

# View the first 5 rows
print(gdf.shape)
gdf.head()

(15800, 11)


,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,48,201,341700,3,1500000US482013417003,482013417003,3,BG,588536,2329,"POLYGON ((-95.019 29.646, -95.018 29.646, -95...."
1,48,037,010901,2,1500000US480370109012,480370109012,2,BG,5976489,137294,"POLYGON ((-94.075 33.486, -94.072 33.486, -94...."
2,48,427,950204,2,1500000US484279502042,484279502042,2,BG,3752003,67429,"POLYGON ((-98.967 26.401, -98.966 26.409, -98...."
3,48,113,013101,2,1500000US481130131012,481130131012,2,BG,761546,0,"POLYGON ((-96.788 32.901, -96.788 32.904, -96...."
4,48,245,007100,2,1500000US482450071002,482450071002,2,BG,3473239,20518,"POLYGON ((-94.032 29.902, -94.01 29.902, -94.0..."


In [11]:
# make sure points match the crs of gdf
gdfPoints = gpd.GeoDataFrame(
    dfCrime, 
    geometry='geometry',
    crs="EPSG:4269"  # WGS84 coordinate reference system
)
gdfPoints.head()

,NIBRSDescription,geometry
6,Aggravated Assault,POINT (-95.441 29.676)
16,Aggravated Assault,POINT (-95.588 29.705)
23,Aggravated Assault,POINT (-95.314 29.671)
32,Robbery,POINT (-95.413 29.731)
33,Robbery,POINT (-95.506 29.815)


In [13]:
# Basic spatial join (points within polygons)
result = gpd.sjoin(
    gdfPoints, 
    gdf,
    how="left",       # 'left', 'right', or 'inner'
    predicate="within" # Spatial relationship
)
print(result.shape)
result.head()

(24318, 13)


,NIBRSDescription,geometry,index_right,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER
6,Aggravated Assault,POINT (-95.441 29.676),6602.0,48,201,420200,2,1500000US482014202002,482014202002,2,BG,459128.0,0.0
16,Aggravated Assault,POINT (-95.588 29.705),840.0,48,201,452500,2,1500000US482014525002,482014525002,2,BG,2036350.0,0.0
23,Aggravated Assault,POINT (-95.314 29.671),3835.0,48,201,332600,3,1500000US482013326003,482013326003,3,BG,1682535.0,0.0
32,Robbery,POINT (-95.413 29.731),8479.0,48,201,411000,3,1500000US482014110003,482014110003,3,BG,661527.0,0.0
33,Robbery,POINT (-95.506 29.815),3573.0,48,201,521300,3,1500000US482015213003,482015213003,3,BG,970313.0,0.0


In [15]:
crime = result['AFFGEOID'].value_counts()
crime

AFFGEOID
1500000US482013125001    171
1500000US482013341002    126
1500000US482011000003    119
1500000US482013333002    116
1500000US482014336003    113
                        ... 
1500000US482015517031      1
1500000US481677201001      1
1500000US482014115013      1
1500000US482015108005      1
1500000US481677212012      1
Name: count, Length: 1417, dtype: int64

In [17]:
# export as a csv
crime.to_csv('Houston_Violent_Crime_BG.csv')